# 순환신경망 (RNN)
 - 같은 요소가 순서지어 반복해서 나타나는 데이터
  - 소리 
  - 자연어 
  - 동영상 

![img1](img/1.PNG)
 - 영향이 여러곳으로 갈 수 있다는 부분이다.
 - 안녕하세요가 안이 뒤쪽으로 가면서도 영향을 미치고 녕도 뒤로가면서 영향을 끼치는 모냥

![img2](img/2.PNG)

## 실습
 - 컴퓨터에게 2자리 수 덧셈을 시킬 것이다. 
 - 47 + 21 = 68 

In [1]:
import random

from keras.layers import Activation, Dense, LSTM, RepeatVector
from keras.layers.wrappers import TimeDistributed
from keras.models import Sequential
import numpy

Using TensorFlow backend.


In [2]:
DIGITS = 2 # 2자리 숫자라서 Digits 2
MAX_LEN = 2 * DIGITS + 1 # 47 + 21 5자리 최대.
NUM_CHAR = len('0123456789+') # 나올 수 있는 글자의 종류 

In [3]:
MAX_LEN

5

In [4]:
NUM_CHAR

11

In [5]:
N = 10000

In [6]:
x = numpy.zeros((N, MAX_LEN, NUM_CHAR), dtype=numpy.bool)
y = numpy.zeros((N, DIGITS + 1, NUM_CHAR), dtype=numpy.bool)

 - 덧셈을 one-hot encoding하기 위해서 
 - 47 + 21 의 경우 
  - 0 0 0 0 1 0 0 0 0 0 0 
  - 0 0 0 0 0 0 0 1 0 0 0
  - 0 0 0 0 0 0 0 0 0 0 1
  - 0 0 1 0 0 0 0 0 0 0 0 
  - 0 1 0 0 0 0 0 0 0 0 0  
 - 위와 같은 형태로 변환 

In [7]:
x.shape # 5개 문자가 들어가는데 각 해당 되는 One-hot Encoding이 11개.

(10000, 5, 11)

In [8]:
y.shape # 결과값이 3자리 라서 

(10000, 3, 11)

In [9]:
x

array([[[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       ..., 
       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [Fal

In [10]:
y

array([[[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       ..., 
       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        

In [11]:
for i in range(N):
    a = random.randint(1, 99)
    b = random.randint(1, 99)
    if i % 2000 == 0:
        print("a :" + str(a))
        print("b :" + str(b))
        print("a 의 몫 : " + str(a // 10))
        print("a 의 나머지 : " + str(a % 10))
        print("b 의 몫 : " + str(b // 10))
        print("b 의 나머지 : " + str(b % 10))
        
    x[i, 0, a // 10] = True # 몫
    x[i, 1, a % 10] = True # 나머지
    x[i, 2, 10] = True # + 자리에 True
    x[i, 3, b // 10] = True # 몫
    x[i, 4, b % 10] = True # 나머지.
    
    y[i, 0, (a + b) // 100] = True # 실제 정답
    y[i, 1, (a + b) % 100 // 10] = True # 몫
    y[i, 2, (a + b) % 10] = True # 나머지

a :71
b :92
a 의 몫 : 7
a 의 나머지 : 1
b 의 몫 : 9
b 의 나머지 : 2
a :30
b :75
a 의 몫 : 3
a 의 나머지 : 0
b 의 몫 : 7
b 의 나머지 : 5
a :72
b :42
a 의 몫 : 7
a 의 나머지 : 2
b 의 몫 : 4
b 의 나머지 : 2
a :16
b :78
a 의 몫 : 1
a 의 나머지 : 6
b 의 몫 : 7
b 의 나머지 : 8
a :22
b :21
a 의 몫 : 2
a 의 나머지 : 2
b 의 몫 : 2
b 의 나머지 : 1


### 번역기를 만든다면.
 - 위의 숫자 자리에 단어라고 생각하고 TDM형태로 만들면 될 것이다. 

In [12]:
HIDDEN = 128 #  LSM의 Hidden을 몇개를 할 것인지?
# return_sequenceses 가 ture가 ㅇ니면 답이 안나간다 1자리만 나간다 

In [13]:
import numpy as np

In [15]:
model = Sequential()
model.add(LSTM(HIDDEN, input_shape=(MAX_LEN, NUM_CHAR)))
model.add(RepeatVector(DIGITS + 1))
model.add(LSTM(HIDDEN, activation='relu', return_sequences=True))
model.add(LSTM(HIDDEN, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(NUM_CHAR)))
model.add(Activation('softmax'))

In [16]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               71680     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 3, 128)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 3, 128)            131584    
_________________________________________________________________
lstm_3 (LSTM)                (None, 3, 128)            131584    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 3, 11)             1419      
_________________________________________________________________
activation_1 (Activation)    (None, 3, 11)             0         
Total params: 336,267
Trainable params: 336,267
Non-trainable params: 0
_________________________________________________________________


 - MNIST처럼 맞고 틀리고를 봐야하므로 categorical_crossenytopy로 사용.

In [17]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [18]:
model.fit(x, y, batch_size=100, epochs=50, verbose=1)

Epoch 1/50
10000/10000 [==============================] - 5s - loss: 1.9883 - acc: 0.2408     
Epoch 2/50
10000/10000 [==============================] - 2s - loss: 1.7183 - acc: 0.3185     
Epoch 3/50
10000/10000 [==============================] - 2s - loss: 1.6694 - acc: 0.3426     
Epoch 4/50
10000/10000 [==============================] - 2s - loss: 1.4993 - acc: 0.4186     
Epoch 5/50
10000/10000 [==============================] - 2s - loss: 1.0898 - acc: 0.5765     
Epoch 6/50
10000/10000 [==============================] - 2s - loss: 1.0314 - acc: 0.5922     
Epoch 7/50
10000/10000 [==============================] - 2s - loss: 0.9535 - acc: 0.6285     
Epoch 8/50
10000/10000 [==============================] - 2s - loss: 0.9198 - acc: 0.6412     
Epoch 9/50
10000/10000 [==============================] - 2s - loss: 0.9054 - acc: 0.6473     
Epoch 10/50
10000/10000 [==============================] - 2s - loss: 0.8837 - acc: 0.6576     
Epoch 11/50
10000/10000 [========================

In [20]:
p = model.predict(x[0:20])

In [21]:
for i in range(20):
    print('x', numpy.argmax(x[i, 0]), numpy.argmax(x[i, 1]), numpy.argmax(x[i, 3]), numpy.argmax(x[i, 4]))
    print('y', numpy.argmax(y[i, 0]), numpy.argmax(y[i, 1]), numpy.argmax(y[i, 2]))
    print('p', numpy.argmax(p[i, 0]), numpy.argmax(p[i, 1]), numpy.argmax(p[i, 2]))
    print('-' * 30)

x 7 1 9 2
y 1 6 3
p 1 6 3
------------------------------
x 4 3 2 4
y 0 6 7
p 0 6 7
------------------------------
x 6 9 1 5
y 0 8 4
p 0 8 4
------------------------------
x 9 1 9 7
y 1 8 8
p 1 8 8
------------------------------
x 7 2 6 3
y 1 3 5
p 1 3 5
------------------------------
x 8 1 1 3
y 0 9 4
p 0 9 4
------------------------------
x 6 5 9 8
y 1 6 3
p 1 6 3
------------------------------
x 1 9 0 3
y 0 2 2
p 0 2 2
------------------------------
x 0 1 9 2
y 0 9 3
p 0 9 3
------------------------------
x 5 3 4 1
y 0 9 4
p 0 9 4
------------------------------
x 3 6 7 2
y 1 0 8
p 1 0 8
------------------------------
x 7 7 1 6
y 0 9 3
p 0 9 3
------------------------------
x 1 5 7 2
y 0 8 7
p 0 8 7
------------------------------
x 2 2 8 7
y 1 0 9
p 1 0 9
------------------------------
x 8 6 4 6
y 1 3 2
p 1 3 2
------------------------------
x 8 5 3 6
y 1 2 1
p 1 2 1
------------------------------
x 9 9 8 3
y 1 8 2
p 1 8 2
------------------------------
x 4 5 6 7
y 1 1 2
p 1 1 2
-----